In [1]:
using Flux

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /home/edoardo/.julia/packages/CUDAdrv/mCr0O/src/CUDAdrv.jl:69


In [2]:
mutable struct MinimalRNNCell
    kernel
    recurrent_kernel
    hidden
end

function MinimalRNNCell(input_shape, units::Integer)
    @show input_shape, units
    kernel = rand(input_shape[end], units)*0.1.-0.05
    recurrent_kernel = rand(units, units)*0.1.-0.05
    MinimalRNNCell(kernel, recurrent_kernel, zeros(units))
end

function (c::MinimalRNNCell)(hᵢ₋₁::AbstractArray, x::AbstractArray)
    output = (c.kernel * x) + (c.recurrent_kernel * hᵢ₋₁)
    c.hidden = output
    return output, output
end
Flux.hidden(c::MinimalRNNCell) = c.hidden

Flux.@treelike MinimalRNNCell

In [3]:
function GenerateSample(length, output)
    inputs = randn(length, output) * 0.1
    outputs = zeros((length,output))
    for i in 1:length
        if i == 1
            outputs[i,:] = inputs[i,:]
        else
            outputs[i,:] = inputs[i,:] + inputs[i-1,:] 
        end
    end
    inputs, outputs
end

GenerateSample (generic function with 1 method)

In [4]:
train_x = zeros(50,10,3)
train_y = zeros(50,10,3)
for i in 1:50
   train_x[i,:,:], train_y[i,:,:] = GenerateSample(10,3) 
end

In [5]:
opt = Flux.ADAM()
function loss(model, x,y)
    total_loss = 0.
    for i in 1:size(x,1)
        for j in 1:size(x,2)
            total_loss += Flux.mse(model(train_x[i,j,:]),y[i,j,:])
        end    
        Flux.reset!(model)
    end
    total_loss / size(x,1)
end
m = Chain(Flux.Recur(MinimalRNNCell((10,3),3)))

(input_shape, units) = ((10, 3), 3)


Chain(Recur(MinimalRNNCell([-0.00206949 -0.0243138 0.0280359; 0.0180573 -0.0376258 0.00967512; 0.0368002 -0.0135777 0.0473345], [0.0478481 0.0272412 -0.024224; 0.00323769 -0.0459578 0.0263321; 0.0334654 0.0322919 -0.0205755], [0.0, 0.0, 0.0])))

In [29]:
function fit!(model, train_x, train_y, epochs; opt=Flux.ADAM(), ps=params(m) )
    Flux.reset!(model)
    for i in 1:epochs
        println("Epoch $i")
         start = time_ns()
        gs = gradient(ps) do
            loss(model, train_x, train_y)
        end
        Flux.Optimise.update!(opt, ps, gs)
        stop = time_ns()
        println("$((stop-start) / 1000 ) μs")
    end
end

fit! (generic function with 1 method)

In [32]:
fit!(m, train_x, train_y, 10)

Epoch 1
111516.723 μs
Epoch 2
76659.385 μs
Epoch 3
72747.869 μs
Epoch 4
82901.475 μs
Epoch 5
77223.784 μs
Epoch 6
78871.266 μs
Epoch 7
75633.205 μs
Epoch 8
76427.501 μs
Epoch 9
78777.9 μs
Epoch 10
76538.759 μs
